In [ ]:
from glob import glob
import xarray as xr
import cftime
import nc_time_axis
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm
from dask_gateway import Gateway
import dask

#! pip install cmip6_preprocessing
from cmip6_preprocessing.preprocessing import (correct_units,rename_cmip6)

In [ ]:
def launchDask():
    #TODO try, except
    gateway = Gateway()
    ## set opts ##
    options = gateway.cluster_options()
    options.worker_memory=14
    options.worker_cores=2
    ##
    clusters = gateway.list_clusters()
    print(clusters)
    if len(clusters) == 1:
        cluster = gateway.connect(clusters[0].name)
        print("shutdown existing connection",clusters[0].name )#or shut down and start afresh cluster.shutdown(). otherwise I dunno how to apply custom options
        cluster.shutdown()
        cluster = gateway.new_cluster(options)
    elif (len(clusters)) > 1:
        print("check open cluster connections and then proceed")
    else:
        cluster = gateway.new_cluster(options)
    return(cluster)


cluster = launchDask()

cluster.scale(20)
client = cluster.get_client()
dask.config.set({"distributed.comm.timeouts.tcp": "50"})

In [ ]:
client

In [ ]:
# Pass this function for preprocessing thetao data 
# Necessary for creating a common time axis for all models
# We want to create a common time axis so there will be no gaps when plotting the results 

def fix_time(ds):
    """ force calendar to noleap"""
    import xarray as xr
    
    if "time" not in ds.dims:
        return ds
    
    if ("calendar" not in ds["time"].attrs): 
        ds["time"].attrs.update({"calendar": "noleap"})
        
    if ds["time"].attrs["calendar"] not in ["noleap", "NOLEAP", "365_day"]:
        ds["time"].attrs.update({"calendar": "noleap"})
        
    ds = xr.decode_cf(ds)
    return ds

def pp_thetao(ds):
    ds = rename_cmip6(ds)
    ds = fix_time(ds)
    ds = correct_units(ds)
    return ds

In [5]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json"
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data
col

,unique
project,1
institute,38
model,107
experiment_id,217
frequency,2
modeling_realm,3
mip_table,37
ensemble_member,2726
grid_label,13
variable,737


In [6]:
# Search for what you want by creating a query 
gr_models = ['GISS-E2-1-H']#,'GFDL-CM4']#,'GFDL-ESM4']#,'CESM2','CESM2-FV2','CESM2-WACCM-FV2','E3SM-1-1-ECA',

OHC_query = dict(
    variable=['thetao'],
     experiment_id=["historical"],
     mip_table=["Omon"],
     grid_label=['gr'],
     ensemble_member="r1i1p1f1",
     model = gr_models
)

cat_T_gr = col.search(**OHC_query)

cat_T_gr.df.groupby(['model','variable']).nunique()



,,project,institute,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,temporal subset,version,path
model,variable,,,,,,,,,,,
GISS-E2-1-H,thetao,1,1,1,1,1,1,1,1,9,1,9


In [ ]:
dset_dict_T = cat_T_gr.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1}},
                               storage_options={'anon':True}, preprocess = pp_thetao)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'
